# Commonsense QA: Baselines

EECS 595 Final Project, Task 1: Commonsense QA

Credit: Ziqiao Ma

Last update: 2020.12.1

# Setup

## Colab setups

Run this cell load the autoreload extension.

In [1]:
%load_ext autoreload
%autoreload 2

Run the following cell to mount your Google Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fill in the Google Drive path where you uploaded the file.

In [3]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/eecs595/commonsense_qa'

Test if files are located.

In [4]:
import os
import sys

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)
print(os.listdir(GOOGLE_DRIVE_PATH))

['roberta.ipynb', 'csqa-graph-reasoning.ipynb', 'csqa-baseline.ipynb']


## Dependency installation

In [5]:
import json
import codecs
import argparse
from copy import deepcopy
from tqdm import tqdm, trange

import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Install `datasets`

In [6]:
!pip install datasets
from datasets import load_dataset

     |████████████████████████████████| 163kB 8.5MB/s 
     |████████████████████████████████| 17.7MB 212kB/s 
     |████████████████████████████████| 245kB 54.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Install `sentencepiece` for `XLNetTokenizer`

In [7]:
!pip install sentencepiece
import sentencepiece

     |████████████████████████████████| 1.1MB 8.4MB/s 


Install `transformers`

In [8]:
!pip install transformers
# !pip install transformers==2.0.0

from transformers import (AdamW, get_linear_schedule_with_warmup, AutoModelForQuestionAnswering,
                          BertConfig, BertForMultipleChoice, BertTokenizer,
                          XLNetConfig, XLNetForMultipleChoice, XLNetTokenizer,
                          RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer,
                          GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer)

     |████████████████████████████████| 1.4MB 8.6MB/s 
     |████████████████████████████████| 890kB 29.5MB/s 
     |████████████████████████████████| 2.9MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9ba74622df8c37be03d198e5c00dea8ba139105e47f4f62c2972e63dd49fe7a3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Helper Functions

In [9]:
SEED = 0

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def load_model(model='all'):
    if model == 'bert':
        return BertConfig, BertForMultipleChoice, BertTokenizer
    elif model == 'xlnet':
        return XLNetConfig, XLNetForMultipleChoice, XLNetTokenizer
    elif model == 'roberta':
        return RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer
    elif model == 'gpt2':
        raise NotImplemented
        # return GPT2Config, AutoModelForQuestionAnswering, GPT2Tokenizer
    raise NotImplemented


def load_optimizer(args, model, train_size):
    num_training_steps = train_size // args.num_train_epochs
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0}
    ]

    optimizer = AdamW(
        optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=num_training_steps)

    return model, optimizer, scheduler

# Benchmark

## Dataset

As a question answering benchmark, [Commonsense QA](https://arxiv.org/abs/1811.00937) presents a natural language question $Q$ of $m$ tokens $\{q_1,q_2,\cdots,q_m\}$ and 5 choices $\{a_1,a_2,\cdots,a_5\}$ labeled with $\{A,B,\cdots,E\}$ regarding each question [1]. Notably, the questions do not entail a inference basis in themselves, so the lack of evidence requires the model to hold a comprehensive understanding on common sense knowledge and a strong reasoning ability to make the right choice.

In [10]:
def load_data(dataset='commonsense_qa', preview=-1):

    assert dataset in {'commonsense_qa', 'conv_entail', 'eat'}

    if dataset == 'commonsense_qa':
        ds = load_dataset('commonsense_qa')

        if preview > 0:
            print('\nLoading an example...')
            data_tr = ds.data['train']
            question = data_tr['question']
            choices = data_tr['choices']
            answerKey = data_tr['answerKey']
            print(question[preview])
            for label, text in zip(choices[preview]['label'], choices[preview]['text']):
                print(label, text)
            print('Ans:', answerKey[preview])

    elif dataset == 'conv_entail':
        dev_file = '/content/drive/Shareddrives/EECS595-Fall2020/Final_Project_Common/Conversational_Entailment/dev_set.json'
        act_file = '/content/drive/Shareddrives/EECS595-Fall2020/Final_Project_Common/Conversational_Entailment/act_tag.json'
        dev_set = codecs.open(dev_file, 'r', encoding='utf-8').read()
        act_tag = codecs.open(act_file, 'r', encoding='utf-8').read()
        ds = json.loads(dev_set), json.loads(act_tag)

        if preview > 0:
            print('Preview not yet implemented for this dataset.')

    else:
        file_name = '/content/drive/Shareddrives/EECS595-Fall2020/Final_Project_Common/EAT/eat_train.json'
        eat = codecs.open(file_name, 'r', encoding='utf-8').read()
        ds = json.loads(eat)

        if preview > 0:
            print('\nLoading an example...')
            story = ds[preview]['story']
            label = ds[preview]['label']
            bp = ds[preview]['breakpoint']
            for line in story:
                print(line)
            print(label)
            print(bp)

    return ds

Run the following code to preview the dataset:

In [11]:
ds = load_data(dataset='commonsense_qa', preview=5)
print('\nDataset statistics:')
print(ds)

Using custom data configuration default


Dataset commonsense_qa downloaded and prepared to /root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29. Subsequent calls will reuse this data.

Loading an example...
What home entertainment equipment requires cable?
A radio shack
B substation
C cabinet
D television
E desk
Ans: D

Dataset statistics:
DatasetDict({
    train: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 1140
    })
})


## Data Preprocessing

In [12]:
class InputExample(object):
    """
    A single multiple choice question.
    """

    def __init__(self, example_id, question, answers, label):
        self.example_id = example_id
        self.question = question
        self.answers = answers
        self.label = label


class InputFeatures(object):
    """
    A single feature converted from an example.
    """

    def __init__(self, example_id, choices_features, label):
        self.example_id = example_id
        self.label = label
        self.choices_features = [
            {'input_ids': input_ids, 'input_mask': input_mask, 'segment_ids': segment_ids}
            for _, input_ids, input_mask, segment_ids in choices_features
        ]


class CommonsenseQAProcessor:
    """
    A Commonsense QA Data Processor
    """

    def __init__(self):
        self.dataset = None
        self.labels = [0, 1, 2, 3, 4]
        self.LABELS = ['A', 'B', 'C', 'D', 'E']

    def get_split(self, split='train'):
        if self.dataset is None:
            self.dataset = load_data(dataset='commonsense_qa', preview=-1)
        return self.dataset[split]

    def create_examples(self, split='train'):
        examples = []
        data_tr = self.get_split(split)
        example_id = 0

        for question, choices, answerKey in zip(data_tr['question'], data_tr['choices'], data_tr['answerKey']):
            answers = np.array(choices['text'])
            label = self.LABELS.index(answerKey)
            examples.append(InputExample(
                example_id=example_id, question=question,
                answers=answers, label=label
            ))
            example_id += 1

        return examples


def truncate_seq_pair(tokens_a, tokens_b, max_length):
    """
    Truncates a sequence pair in place to the maximum length.

    This is a simple heuristic which will always truncate the longer sequence one token at a time.
    This makes more sense than truncating an equal percent of tokens from each,
    since if one sequence is very short then each token that's truncated
    likely contains more information than a longer sequence.

    However, since we'd better not to remove tokens of options and questions,
    you can choose to use a bigger length or only pop from context
    """

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            warning = 'Attention! you are removing from token_b (swag task is ok). ' \
                      'If you are training ARC and RACE (you are popping question + options), ' \
                      'you need to try to use a bigger max seq length!'
            print(warning)
            tokens_b.pop()


def examples_to_features(examples, label_list, max_seq_length, tokenizer,
                         cls_token_at_end=False,
                         cls_token='[CLS]',
                         cls_token_segment_id=1,
                         sep_token='[SEP]',
                         sequence_a_segment_id=0,
                         sequence_b_segment_id=1,
                         sep_token_extra=False,
                         pad_token_segment_id=0,
                         pad_on_left=False,
                         pad_token=0,
                         mask_padding_with_zero=True):
    """
    Convert Commonsense QA examples to features.

    The convention in BERT is:
    (a) For sequence pairs:
    tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1

    (b) For single sequences:
    tokens:   [CLS] the dog is hairy . [SEP]
    type_ids:   0   0   0   0  0     0   0

    Where "type_ids" are used to indicate whether this is the first sequence or the second sequence.
    The embedding vectors for `type=0` and `type=1` were learned during pre-training
    and are added to the word piece embedding vector (and position vector).
    This is not *strictly* necessary since the [SEP] token unambiguously separates the sequences,
    but it makes it easier for the model to learn the concept of sequences.

    For classification tasks, the first vector (corresponding to [CLS]) is used as as the "sentence vector".
    Note that this only makes sense because the entire model is fine-tuned.
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in tqdm(enumerate(examples), desc="Converting examples to features", disable=True):

        choices_features = []
        for ending_idx, (question, answers) in enumerate(zip(example.question, example.answers)):

            tokens_a = tokenizer.tokenize(example.question)
            if example.question.find("_") != -1:
                tokens_b = tokenizer.tokenize(example.question.replace("_", answers))
            else:
                tokens_b = tokenizer.tokenize(answers)

            special_tokens_count = 4 if sep_token_extra else 3
            truncate_seq_pair(tokens_a, tokens_b, max_seq_length - special_tokens_count)

            tokens = tokens_a + [sep_token]
            if sep_token_extra:
                tokens += [sep_token]

            segment_ids = [sequence_a_segment_id] * len(tokens)

            if tokens_b:
                tokens += tokens_b + [sep_token]
                segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens.
            # Only real tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding_length = max_seq_length - len(input_ids)

            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids

            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = label_map[example.label]

        if ex_index < 0:
            print("*** Example ***")
            print("race_id: {}".format(example.example_id))
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                print("choice: {}".format(choice_idx))
                print("tokens: {}".format(' '.join(tokens)))
                print("input_ids: {}".format(' '.join(map(str, input_ids))))
                print("input_mask: {}".format(' '.join(map(str, input_mask))))
                print("segment_ids: {}".format(' '.join(map(str, segment_ids))))
                print("label: {}".format(label))

        features.append(InputFeatures(
            example_id=example.example_id,
            choices_features=choices_features,
            label=label
        ))

    return features


def load_features(args, tokenizer, mode='train'):
    """
    Load the processed Commonsense QA dataset
    """

    def select_field(feature_list, field_name):
        return [
            [choice[field_name] for choice in feature.choices_features]
            for feature in feature_list
        ]

    assert mode in {'train', 'validation', 'test'}
    print("Creating features from dataset...")

    processor = CommonsenseQAProcessor()
    label_list = processor.labels
    examples = processor.create_examples(split=mode)

    print("Training number:", str(len(examples)))
    features = examples_to_features(examples, label_list, args.max_seq_length, tokenizer,
                                    cls_token_at_end=bool(args.model_type in ['xlnet']),
                                    cls_token=tokenizer.cls_token,
                                    sep_token=tokenizer.sep_token,
                                    sep_token_extra=bool(args.model_type in ['roberta']),
                                    cls_token_segment_id=2 if args.model_type in ['xlnet'] else 0,
                                    pad_on_left=bool(args.model_type in ['xlnet']),
                                    pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(select_field(features, 'input_ids'), dtype=torch.long)
    all_input_mask = torch.tensor(select_field(features, 'input_mask'), dtype=torch.long)
    all_segment_ids = torch.tensor(select_field(features, 'segment_ids'), dtype=torch.long)
    all_label_ids = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

# Experiment

## Pre-trained Model

In [13]:
def train(args, model, tokenizer):

    print('\n Loading training dataset')
    dataset_tr = load_features(args, tokenizer, mode='train')
    sampler_tr = RandomSampler(dataset_tr)
    dataloader_tr = DataLoader(dataset_tr, sampler=sampler_tr, batch_size=args.batch_size)

    print('\n Loading validation dataset')
    dataset_val = load_features(args, tokenizer, 'validation')
    sampler_val = SequentialSampler(dataset_val)
    dataloader_val = DataLoader(dataset_val, sampler=sampler_val, batch_size=args.batch_size)

    model, optimizer, scheduler = load_optimizer(args, model, len(dataloader_tr))

    num_steps = 0
    best_steps = 0
    tr_loss = 0.0
    best_val_acc, best_val_loss = 0.0, 99999999999.0
    best_model = None

    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=False, leave=True, position=1)

    for _ in train_iterator:

        epoch_iterator = tqdm(dataloader_tr, desc="Iteration", disable=False, leave=True, position=1)
        for step, batch in enumerate(epoch_iterator):

            model.train()

            batch = tuple(b.to(args.device) for b in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
                      'labels': batch[3]}
            outputs = model(**inputs)
            loss = outputs[0]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()

            optimizer.step()
            scheduler.step()
            model.zero_grad()
            num_steps += 1

            if args.logging_steps > 0 and num_steps % args.logging_steps == 0:
                results = evaluate(args, model, dataloader_val)
                print("\n val acc: {}, val loss: {}"
                      .format(str(results['val_acc']), str(results['val_loss'])))
                if results["val_acc"] > best_val_acc:
                    best_val_acc, best_val_loss = results["val_acc"], results["val_loss"]
                    best_steps = num_steps
                    best_model = deepcopy(model)

    loss = tr_loss / num_steps

    return best_model


def evaluate(args, model, dataloader):

    val_loss = 0.0
    num_steps = 0
    preds, labels = None, None

    results = {}

    for batch in tqdm(dataloader, desc="Validation", disable=True, leave=True, position=1):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
                      'labels': batch[3]}
            outputs = model(**inputs)
            loss, logits = outputs[:2]

            val_loss += loss.mean().item()

        num_steps += 1

        if preds is None:
            preds = logits.detach().cpu().numpy()
            labels = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            labels = np.append(labels, inputs['labels'].detach().cpu().numpy(), axis=0)

    loss = val_loss / num_steps
    preds = np.argmax(preds, axis=1)
    acc = (preds == labels).mean()
    result = {"val_acc": acc, "val_loss": loss}
    results.update(result)

    return results


def test(args, tokenizer, model):

    dataset = load_features(args, tokenizer, mode='validation')
    sampler = SequentialSampler(dataset)
    dataloader_test = DataLoader(dataset, sampler=sampler, batch_size=args.batch_size)

    results = evaluate(args, model, dataloader_test)
    print('\nTesting...')
    print("\n final validation acc: {}, final validation loss: {}"
          .format(str(results['val_acc']), str(results['val_loss'])))

In [14]:
def main(args):

    print('Using device', args.device)
    set_seed(args.seed)

    processor = CommonsenseQAProcessor()
    num_labels = len(processor.labels)

    config_class, model_class, tokenizer_class = load_model(args.model_type)
    config = config_class.from_pretrained(
        args.config_name if args.config_name else args.model_name,
        num_labels=num_labels, finetuning_task=args.task_name)
    tokenizer = tokenizer_class.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name,
        do_lower_case=True)
    
    model = model_class.from_pretrained(
        args.model_name, from_tf=bool('.ckpt' in args.model_name), config=config)
    model.to(args.device)

    best_model = train(args, model, tokenizer)
    test(args, tokenizer, best_model)

## Runtime

The default model is based on `bert`, and
```
parser.add_argument("--model_name", type=str, default='bert-base-uncased',
                    help="Path to pre-trained model or shortcut name. See https://huggingface.co/models")
```

This would leads to

```
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using custom data configuration default
```

Should check https://huggingface.co/models for other models.

In [16]:
def run(model_type='bert',
        model_name='bert-base-uncased',
        task_name=None,
        batch_size=16,
        logging_steps=50,
        lr=1e-5,
        epochs=1):
  
    parser = argparse.ArgumentParser(description="Common sense question answering")
    parser.add_argument("--model_type", type=str, default=model_type,
                        help="Model: <str> [ bert | xlnet | roberta | gpt2 ]")
    parser.add_argument("--task_name", default=task_name, type=str, required=False,
                        help="The name of the task to train: <str> [ commonqa ]")
    parser.add_argument("--model_name", type=str,
                        default=model_name,
                        help="Path to pre-trained model or shortcut name."
                              "See https://huggingface.co/models")
    parser.add_argument("--config_name", type=str,
                        default=model_name,
                        help="Pre-trained config name or path")
    parser.add_argument("--tokenizer_name", default=model_name, type=str,
                        help="Pre-trained tokenizer name or path if not the same as model_name")

    parser.add_argument("--max_seq_length", default=128, type=int,
                        help="The maximum total input sequence length after tokenization. "
                                "Sequences longer than this will be truncated, sequences shorter will be padded.")
    parser.add_argument("--batch_size", default=batch_size, type=int,
                        help="Batch size for training.")

    parser.add_argument("--learning_rate", default=lr, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-6, type=float,
                        help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")

    parser.add_argument("--num_train_epochs", default=epochs, type=int,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--warmup_steps", default=10, type=int,
                        help="Linear warmup over warmup_steps.")
    parser.add_argument('--logging_steps', type=int, default=logging_steps,
                        help="Log every n updates steps.")

    parser.add_argument('--fp16', type=bool, default=True,
                        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
    parser.add_argument('--opt_level', type=str, default='O1',
                        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                             "See details at https://nvidia.github.io/apex/amp.html")

    parser.add_argument("--seed", type=int, default=SEED, help="Random seed: <int>")
    parser.add_argument("--device", default=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    args, unknown = parser.parse_known_args()
    main(args)

## Finetuning

### Bert

In [ ]:
model_type = 'bert'
model_name = 'bert-base-uncased'
task_name = 'commonqa'
batch_size = 16
logging_steps = 100
lr = 5e-5

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly


 Loading training dataset
Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 9741


...


 val acc: 0.5618345618345618, val loss: 1.0950990009617496



Iteration: 100%|██████████| 609/609 [22:05<00:00,  2.18s/it]

Epoch: 100%|██████████| 1/1 [22:05<00:00, 1325.43s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.5659295659295659, final validation loss: 1.1136866111259955


In [ ]:
model_type = 'bert'
model_name = 'bert-large-uncased'
task_name = 'commonqa'
batch_size = 4
logging_steps = 400
lr = 1e-5

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-large-uncased and are new


 Loading training dataset
Creating features from dataset...
Training number: 9741


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)



 Loading validation dataset
Creating features from dataset...
Training number: 1221


...


 val acc: 0.6167076167076168, val loss: 0.9998837094875722



Iteration: 100%|██████████| 2436/2436 [1:13:14<00:00,  1.80s/it]

Epoch: 100%|██████████| 1/1 [1:13:14<00:00, 4394.09s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.6167076167076168, final validation loss: 0.9998837094875722


In [ ]:
model_type = 'bert'
model_name = 'deepset/bert-large-uncased-whole-word-masking-squad2'
task_name = 'commonqa'
batch_size = 2
logging_steps = 800
lr = 5e-6

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 were not used when initializing BertForMultipleChoice: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Loading training dataset
Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 9741


...


 val acc: 0.592956592956593, val loss: 1.0395029854149833



Iteration: 100%|██████████| 4871/4871 [1:24:19<00:00,  1.04s/it]

Epoch: 100%|██████████| 1/1 [1:24:19<00:00, 5059.05s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.592956592956593, final validation loss: 1.0395029854149833


### XLNet

In [ ]:
model_type = 'xlnet'
model_name = 'xlnet-base-cased'
task_name = 'commonqa'
batch_size = 4
logging_steps = 400
lr = 1e-6

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultipleChoice: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using custom data 


 Loading training dataset
Creating features from dataset...
Training number: 9741


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)



 Loading validation dataset
Creating features from dataset...
Training number: 1221


...


 val acc: 0.2375102375102375, val loss: 1.606173419874478



Iteration: 100%|██████████| 2436/2436 [31:01<00:00,  1.31it/s]

Epoch: 100%|██████████| 1/1 [31:01<00:00, 1861.33s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.26945126945126946, final validation loss: 1.5953456003681508


### RoBerta

In [ ]:
model_type = 'roberta'
model_name = 'roberta-base'
task_name = 'commonqa'
batch_size = 16
logging_steps = 100
lr = 5e-5

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predi


 Loading training dataset
Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 9741


...


 val acc: 0.5389025389025389, val loss: 1.1480596057780377



Iteration: 100%|██████████| 609/609 [22:02<00:00,  2.17s/it]

Epoch: 100%|██████████| 1/1 [22:02<00:00, 1322.57s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.5389025389025389, final validation loss: 1.1480596057780377


In [ ]:
model_type = 'roberta'
model_name = 'roberta-large'
task_name = 'commonqa'
batch_size = 2
logging_steps = 800
lr = 2e-6

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMultipleChoice: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pre


 Loading training dataset
Creating features from dataset...


Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 9741


...


 val acc: 0.6003276003276004, val loss: 1.0647238510738564



Iteration: 100%|██████████| 4871/4871 [1:19:48<00:00,  1.02it/s]

Epoch: 100%|██████████| 1/1 [1:19:48<00:00, 4788.53s/it]


Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 1221

Testing...

 final validation acc: 0.6003276003276004, final validation loss: 1.0647238510738564


In [ ]:
model_type = 'roberta'
model_name = 'deepset/roberta-base-squad2'
task_name = 'commonqa'
batch_size = 16
logging_steps = 100
lr = 5e-5

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

In [ ]:
model_type = 'roberta'
model_name = 'roberta-large'
task_name = 'commonqa'
batch_size = 2
logging_steps = 800
lr = 1e-5

run(model_type, model_name, task_name, batch_size, logging_steps, lr)

Using device cuda


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMultipleChoice: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pre


 Loading training dataset
Creating features from dataset...


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


Training number: 9741

 Loading validation dataset
Creating features from dataset...


...


Iteration:  16%|█▋        | 800/4871 [12:57<48:57:15, 43.29s/it]


 val acc: 0.18263718263718265, val loss: 1.6094383553475289



Iteration:  33%|███▎      | 1600/4871 [26:08<39:12:01, 43.14s/it]


 val acc: 0.20884520884520885, val loss: 1.6094372887072899



Iteration:  35%|███▍      | 1697/4871 [27:26<42:55,  1.23it/s]


# Results

## Validation Accuracy

The validation accuracy without finetuning is listed as follows:
* Bert:  56.6% (base), 61.7% (large)
* XLNet: 26.9%
* RoBerta: 53.9% (base), 60.0% (large)

## Todo

A few things to do in the future:
1. Further tune `xlnet`.
2. Run `roberta-large` with more epochs. Currently `epochs = 1`.
3. Finish all finetuning experiments.

# References

[1] Talmor, A., Herzig, J., Lourie, N., & Berant, J. (2019). CommonsenseQA: A Question Answering Challenge Targeting Commonsense Knowledge. ArXiv, abs/1811.00937.